[ ] 跑通contriever+faiss+llama7b+nq

In [3]:
# Create conda environment

!conda create -n 5018rag python=3.9 --yes
!conda activate 5018rag

Channels:
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/goodh/miniconda3/envs/5018rag

  added / updated specs:
    - python=3.9


The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main 
  _openmp_mutex      pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu 
  ca-certificates    pkgs/main/linux-64::ca-certificates-2024.9.24-h06a4308_0 
  ld_impl_linux-64   pkgs/main/linux-64::ld_impl_linux-64-2.40-h12ee557_0 
  libffi             pkgs/main/linux-64::libffi-3.4.4-h6a678d5_1 
  libgcc-ng          pkgs/main/linux-64::libgcc-ng-11.2.0-h1234567_1 
  libgomp            pkgs/main/linux-64::libgomp-11.2.0-h1234567_1 
  libstdcxx-ng       pkgs/main/linux-64::libstdcxx-ng-11.2.0-h1234567_1 
  ncurses            pkgs/main/linux-64::ncurses-6.4-h6a678d5_0 
  openssl            pkgs/main/linux-64::openssl-3.0.15-h5eee18b_0 
  pip                pkgs/main/linux-64::pip-24.2-py39h06a4308_0 

In [1]:
### install requirements

!pip install -r requirements.txt
!pip install datasets

# install faiss  [https://github.com/facebookresearch/faiss/blob/main/INSTALL.md]
!conda install -c pytorch -c nvidia faiss-gpu=1.9.0 --yes

# install huggingface cli [https://huggingface.co/docs/huggingface_hub/guides/cli]
!pip install -U "huggingface_hub[cli]"

!huggingface-cli download --repo-type dataset --resume-download florin-hf/wiki_dump2018_nq_open --local-dir /data/wiki_dump2018_nq_open
!huggingface-cli download --repo-type dataset --resume-download florin-hf/nq_open_gold --local-dir /data/nq_open_gold

Channels:
 - defaults
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Channels:
 - pytorch
 - nvidia
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Channels:
 - pytorch
 - defaults
 - conda-forge
 - nvidia
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/goodh/miniconda3/envs/5018rag

  added / updated specs:
    - pytorch==2.4.1
    - torchaudio==2.4.1
    - torchvision==0.19.1


The following NEW packages will be INSTALLED:

  brotli-python      pkgs/main/linux-64::brotli-python-1.0.9-py39h6a678d5_8 
  bzip2              pkgs/main/linux-64::bzip2-1.0.8-h5eee18b_6 
  certifi            pkgs/main/linux-64::certifi-2024.8.30-py39h06a4308_0 
  charset-normalizer pkgs/main/noarch::charset-normalizer-3.3.2-pyhd3eb1b0_0 
  cuda-cudart_linux~ pkgs/main/noarch::cuda-cudart_linux-64-12.4.127-hd681fbe

In [2]:
from datasets import load_dataset
_corpus = load_dataset('/data/wiki_dump2018_nq_open')
_dataset = load_dataset('/data/nq_open_gold')

import pandas as pd
import json

# save corpus as a json file at location 'data/corpus.json
corpus_df = pd.DataFrame(_corpus['train'].select(range(100, 125))) # keep only 25 corpus and dataset
# corpus_df = pd.DataFrame(_corpus['train'])
corpus_df.to_json('data/corpus.json', orient='records')

dataset = _dataset['train'].select(range(100, 115))

RuntimeError: CPU dispatcher tracer already initlized

ImportError: numpy._core.multiarray failed to import

## 1. Retrieval

### 1. compute_corpus_embeddings ✔️

run "1.1 Compute Corpus Embeddings"

### 2. index_embeddings ✔️

run 1.2 Index Embeddings

### 3. compute_search_results ✔️

run 1.3 Compute Search Results

## 2. Generation

### - w/o RAG ✔️

run 2.1 Generation w/o RAG

### - w/ RAG ✔️

run 2.2 Generation w/ RAG
- used gold + distracting documents, gold doc placed at middle

### - w/ improved RAG

In [ ]:
!conda install pytorch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 -c pytorch --yes # cause datasets error
!conda install pandas==2.1.4 --yes
!conda install transformers --yes

In [ ]:
# 查看 "w/o RAG" 模型回答
import pickle
with open('data/gen_res/Llama-2-7b-chat-hf/train/only_query/only_query_info_9.pkl', 'rb') as f:
    only_query_info = pickle.load(f)
print(only_query_info)

In [1]:
# 查看 "w/ RAG" 模型回答
import pickle
with open('data/gen_res/Llama-2-7b-chat-hf/train/classic/contriever/7_doc/numdoc7_gold_at3_answerless_info_6.pkl', 'rb') as f:
    numdoc7 = pickle.load(f)
print(numdoc7)

[{'example_id': ['-6802534628745605728', '-3481854117164495809', '722054169493956493', '-1491691269352675298', '3124518073686486889', '8830971390439307916', '-4511863004608336817', '-6873948815061621131', '-7905631867159455409', '454496975962020834', '-6339537022235364675', '-4485964469272704887', '-6300616333349284423', '-8703525899468176959', '4968093302052775704', '-4603561353832160748', '-7542477782683626886', '495828222288302243'], 'query': ['total number of death row inmates in the us', 'what is the name of latest version of android', 'who is the sister of for king and country', 'who plays skyler on lab rats elite force', "who played all the carly's on general hospital", 'who is the yellow rose of texas named after', 'who won the nrl grand final in 2015', 'when was the wheel first used in egypt', "when does season 14 of grey's anatomy come out", 'who sings the original version of feeling good', 'who is the chief law officer of the government of india', 'what day of the year is mo

## 3. Evaluation

### - w/o RAG ✔️
run 3.1 Evaluate w/o RAG Result (demo ACC 12%)

### - w/ RAG ✔️
run 3.2 Evaluate w/ RAG Result (demo ACC 31%)

### - w/ improved RAG
run 3.3 Evaluate w/ improved RAG Result